In [ ]:
!pip install labelme tensorflow  opencv-python matplotlib albumentations


!pip list

In [ ]:
#import os
import cv2
import time
import uuid
import tensorflow as tf
import json
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
#os.path.join()

load stuff 

In [ ]:
def load_image(x)
    byte_img=tf.io.read_file(x)
    ig= tf.io.decode_jpeg(byte_img)
    return img


In [ ]:
Plt.imshow(train_images.as_numpy_interator().next())

In [ ]:
train_images = tf.data_dataset.list_files('aug_data\\train\\images\\*.jpg', shuffle=false)
train_images = train_images.map(load_image)
train_images = train_images.map(lambda x:tf.image.resize(x,(250,250) ))
train_images = train_images.map(lambada x:x/255)

In [ ]:
plt.imshow(train_images.as_numpy_interator().next())

In [ ]:
test_images = tf.data_dataset.list_files('aug_data\\test\\images\\*.jpg', shuffle=false)
test_images = test_images.map(load_image)
test_images = test_images.map(lambda x:tf.image.resize(x,(250,250) ))
test_images = test_images.map(lambada x:x/255)

In [ ]:
val_images = tf.data_dataset.list_files('aug_data\\val\\images\\*.jpg', shuffle=false)
val_images = val_images.map(load_image)
val_images = val_images.map(lambda x:tf.image.resize(x,(250,250) ))
val_images = val_images.map(lambada x:x/255)

In [ ]:
test_images.as_numpy_interator().next()

label setting 

In [ ]:
def load_labels(label_path):
    with open(label_path.numpy(), 'r', encoding = "utf-8") as f:
        label = json.load(f)
    return [label['keypoints']]

In [ ]:
train_labels = tf.data.Dataset.list_files('aug_data\\train\\labels\\*.json', shuffle=False)

In [ ]:
iterator = train_labels.as_numpy_iterator()

In [ ]:
iterator.next

In [ ]:
#firt two x=left eye y=righteye
#second two is flip flop

In [ ]:
train_labels = tf.data.Dataset.list_files('aug_data\\train\\labels\\*.json', shuffle=False)
train_labels = train_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.float16]))

In [ ]:
test_labels = tf.data.Dataset.list_files('aug_data\\test\\labels\\*.json', shuffle=False)
test_labels = test_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.float16]))

In [ ]:
val_labels = tf.data.Dataset.list_files('aug_data\\val\\labels\\*.json', shuffle=False)
val_labels = val_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.float16]))

In [ ]:
val_labels.as_numpy_iterator().next()

what the sigma, checking markdownissues 

combine labels with the images

In [ ]:
train = tf.data.Dataset.zip((train_images, train_labels))

In [ ]:
len(train.as_numpy_iterator().next())

In [ ]:
len(train.as_numpy_iterator().next()[0])

In [ ]:
train.as_numpy_iterator().next()[0]

In [ ]:
train = tf.data.Dataset.zip((train_images, train_labels))
train = train.shuffle(5000)
train = train.batch(16)
train = train.prefetch(4)

In [ ]:
test = tf.data.Dataset.zip((test_images, test_labels))
test = test.shuffle(1300)
test = test.batch(16)
test = test.prefetch(4)

viewing the samples

In [ ]:
data_samples = train.as_numpy_iterator()


In [ ]:
res = data_samples.next

In [ ]:
len(res)   

In [ ]:
res[0].shaape

In [ ]:
res[1][0][0]

In [ ]:
plt.imshow(res[0][0])

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx in range(4): 
    sample_image = res[0][idx]
    sample_coords = res[1][0][idx]
    
    cv2.circle(sample_image, tuple(np.multiply(sample_coords[:2], [250,250]).astype(int)), 2, (255,0,0), -1)
    cv2.circle(sample_image, tuple(np.multiply(sample_coords[2:], [250,250]).astype(int)), 2, (0,255,0), -1)
    
    ax[idx].imshow(sample_image)

network 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, Reshape, Dropout
from tensorflow.keras.applications import ResNet152V2

In [ ]:
input_layer = Input(shape=250,250,3)
backbone = ResNet152V2(include_top=False)(input_layer)


#hidden layer here

#output_layer = 4



In [ ]:
cnn4

In [ ]:
backbone.output_shape

In [ ]:
input_layer

In [ ]:
model = Sequential([
    Input(shape=(250,250,3)), 
    ResNet152V2(include_top=False, input_shape=(250,250,3)),
    Conv2D(512, 3, padding='same', activation='relu'),
    Conv2D(512, 3, padding='same', activation='relu'),
    Conv2D(256, 3, 2, padding='same', activation='relu'),
    Conv2D(256, 2, 2, activation='relu'),
    Dropout(0.05),
    Conv2D(4, 2, 2),
    Reshape((4,))
])

In [ ]:
model.summary

optimizer

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, decay=0.0007)
loss = tf.keras.losses.MeanSquaredError()

In [ ]:
model.compile(optimizer, loss)

checkingisdjngfjkl;

In [ ]:
X, y = train.as_numpy_iterator().next()

In [ ]:
X.shape

In [ ]:
scores = model.predict(X)

In [ ]:
scores

In [ ]:
coordinates = model.predict(X)

In [ ]:
coordinates

train the model 

In [ ]:
# Train for 100 epochs
hist = model.fit(train, epochs=10, validation_data=val)

performance chekc

In [ ]:
hist.history

In [ ]:
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val loss')
plt.suptitle('Loss')
plt.legend()
plt.show()

predictions

In [ ]:
test_data = test.as_numpy_iterator()

In [ ]:
test_sample = test_data.next()

In [ ]:
yhat = model.predict(test_sample[0])

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx in range(4): 
    sample_image = test_sample[0][idx]
    sample_coords = yhat[idx]
    
    cv2.circle(sample_image, tuple(np.multiply(sample_coords[:2], [250,250]).astype(int)), 2, (255,0,0), -1)
    cv2.circle(sample_image, tuple(np.multiply(sample_coords[2:], [250,250]).astype(int)), 2, (0,255,0), -1)
    
    ax[idx].imshow(sample_image)

save the thing

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model.save('eyetrackerresnet.h5')

In [ ]:
model = load_model('eyetrackerresnet.h5')

In [ ]:
model = load_model('eyetrackerresnet.h5')

model use

In [ ]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    _ , frame = cap.read()
    
    frame = frame[50:500,50:500,:] 
    rgb_img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resized = cv2.resize(rgb_img, (250,250))
    
    yhat = model.predict(np.expand_dims(resized/255,0))
    sample_coords = yhat[0,:4]
    
    cv2.circle(frame, tuple(np.multiply(sample_coords[:2], [450,450]).astype(int)), 2, (255,0,0), -1)
    cv2.circle(frame, tuple(np.multiply(sample_coords[2:], [450,450]).astype(int)), 2, (0,255,0), -1)
    
    cv2.imshow('EyeTrack', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()